## Long Example 0: Text classification with pipelines

### How samples are split in Scikit-learn $K$-fold CV


In [ ]:
# Import scikit-learn libraries
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_validate, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# import other libraries
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import random

In [ ]:
# create the range 1 to 14
rn = [1,2,3,4,5,6,7,8,9,10,11,12,13,14]
print(rn)

In [ ]:
# to demonstrate how the data are split, we will create 3 and 5 folds. 
# KFold function has to be applied on the data and it returns an 
# location (index) of the train and test samples.
kf5 = KFold(n_splits=5, shuffle=False)
kf3 = KFold(n_splits=3, shuffle=False)

In [ ]:
# The Kfold function retunrs the indices of the data. 
# Our range goes from 1-14 so the index is 0-13
for train_index, test_index in kf3.split(rn):
    print(train_index, test_index)

In [ ]:
# Comment: we have here that n/k is 14/3, which is 4 with a reminder r=2.
# The two reminder elements are assigned to the first two test folds.

In [ ]:
# K=5 folds:
for train_index, test_index in kf5.split(rn):
    print(train_index, test_index)

# Classification 


## Pipelines in scikit-learn (1)

- We spent a lot of time in pre-processing and cleaning data
- Therefore we need to create multipurpose software objects to be used in diffirent situations.
- For this we can use the Pipeline tool in scikit-learn.
- It is composed by *transformers* (tools for transforming data, for example to normalize a variable) and *estimators* (for example a fitting or predicting tool).
- All transformers and estimators in scikit-learn are implemented as Python *classes*, each with their own attributes and methods.
- We use *inherited* classes from scikit-learn to implement our own class.

<img src="Pipeline.png" width="500">

In [ ]:
# Example of inheritance
from sklearn.preprocessing import OneHotEncoder 
#Some data:
X = [[0, 0, 3], [1, 1, 0], [0, 2, 1], [1, 0, 2]]
#Initializing an object of class OneHotEncoder
# Here we are "inheriting" classes from OneHotEncoder into the variable 'one_hot_enc'
one_hot_enc = OneHotEncoder( sparse = True )

#Calling methods on our OneHotEncoder object
one_hot_enc.fit( X ) #returns nothing
transformed_data = one_hot_enc.transform(X).toarray() #returns something
#fit_transformed_data = one_hot_enc.transform( X ) #returns something

In [ ]:
print(pd.DataFrame(X))
#Comments: The first column takes on 2 values, the second 3 and the fourth 4

In [ ]:
print(transformed_data)
#Comments: the first two columns express the binary coding of the first "feature"; 
# the next three columns express the binary coding of the second "feature";
# The next four columns express the binary coding of the third "feature";

## Pipelines in scikit-learn (2)

- Our own trasformer will be formed by inheriting from some other scikit-learn class.
- See a tutorial here https://www.programiz.com/python-programming/class about classes and objects in python and a tutorial here https://www.programiz.com/python-programming/inheritance about inheritance.
- The base classes inherited from scikit-learn are TransformerMixin (https://scikit-learn.org/stable/modules/generated/sklearn.base.TransformerMixin.html) and BaseEstimator (https://scikit-learn.org/stable/modules/generated/sklearn.base.BaseEstimator.html).

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Load Data
pd.set_option('display.max_colwidth', None)
BikeSent = pd.read_csv("BikeMiSentiment2019_UTF-8.csv", sep=';')
BikeSent

In [ ]:
# Rename columns
BikeSent.columns = ["target", "text"]
BikeSent.head()

In [ ]:
# Encode categories
BikeSent['target'] = np.where(BikeSent['target']=='positive',1,0)
BikeSent.head()

In [ ]:
# split the sample in train (used also for cross-validation) + test 
from sklearn.model_selection import train_test_split
X = BikeSent[['text']]
y = BikeSent['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [ ]:
X_train

## Custom Transformers



### Cleaning Text
- We create here our own transformer (which will be a class) inheriting the TransformerMixin and the BaseEstimator classes from scikit-learn

In [ ]:
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

# Custom Transformer (Inheriting from classes)
class CleanText( BaseEstimator, TransformerMixin ):
    
    # Class Constructor 
    # The class constructor is formed by a function with double underscore __ :
    # these are called 'special functions' as they have special meaning.
    # In particular the '__init__' gets called whenever 
    # a new object of that class is instantiated,
    # and are used to initialize all the necessary variables.
    # In this example we initialize the language variable 'lang' with 'English'
    # and pick the SnowballStemmer as the default stemmer.
    def __init__( self, lang = "english"):
        self.lang = lang
        self.stemmer = SnowballStemmer(self.lang)
    
    # The 'fit' method here is used to instantiate the class on the 'self' variable 
    # and return the object itself     
    def fit( self, X, y = None ):
        return self 
    
    # Custom function: this applies the stemmer just created in the '__init__'
    # part to the 'self' variable
    def clean( self, x ):
        words   = [self.stemmer.stem(word) for word in word_tokenize(x.lower()) if word.isalpha() and word not in stopwords.words("english")]
        return " ".join(words)
    
    # Method that describes what we need this transformer to do i.e. cleaning the text
    # in the 'text' column in the data frame.
    # This will be used later on in the usage of the custom transformer 
    # within the pipeline.
    def transform( self, X, y = None ):
        return X["text"].apply(self.clean)

### Feature extraction

In [ ]:
# Custom Transformer: same parts as the previous custom transformer
# This one will be used for feature extraction

class CustomFeatures( BaseEstimator, TransformerMixin ):
    
    # Class Constructor 
    def __init__( self ):
        return
    
    # Return self nothing else to do here    
    def fit( self, X, y = None ):
        return self 
        
    # Method that describes what we need this transformer to do i.e.
    # returning length, digits and punctuations in the 'text' column in data frame
    def transform( self, X, y = None ):
        f           = pd.DataFrame()
        f['len']    = X['text'].str.len()
        f['digits'] = X['text'].str.findall(r'\d').str.len()
        f['punct']  = X['text'].str.findall(r'[^a-zA-Z\d\s:]').str.len()
        return f[['len','digits','punct']]

## Pipeline usage


### Pipeline for data pre processing

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
# FeatureUnion combines two or more pipelines or transformers
# and is very fast!
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import StandardScaler
# Our first pipeline called 'pipe' will be formed by three 'steps' or parts:
# 1)"extract" which in turns is formed through FeatureUnion which put together two parts:
# "terms" (formed by a pipeline with the CleanText() transformer we created above
# and the TfidVectorize text vectorizing transformer from scikit-learn) and "custom" 
# (formed by the CustomFeatures transformer we created above);
# 2) "select", formed by the scikit-learn transformer method "SelectKBest" for feature 
# selection with a chi squared score function;
# 3) "scale", same as 2) using the StandardScaler method from scikit-learn. 
# The whole pipeline will be used as pre-processing task in classifying pipelines.
pipe = Pipeline([("extract", FeatureUnion([("terms", Pipeline([('clean', CleanText()), 
                                                               ('tfidf', TfidfVectorizer())])),
                                           ("custom", CustomFeatures())])),
                 ("select", SelectKBest(score_func = chi2)),
                 ("scale", StandardScaler(with_mean = False))]) 

### Classifier implemented through pipelines: Logistic Model

In [ ]:
# Logistic Model
from sklearn.linear_model import LogisticRegression
pipe_logistic = Pipeline([('pre_process', pipe),
                          ('classify', LogisticRegression(max_iter=10000, tol=0.1, solver='lbfgs'))])

In [ ]:
# Fit on training
pipe_logistic.fit(X_train, y_train)

In [ ]:
# Evaluate on test
# The F1 score can be interpreted as a weighted average of the precision and recall, 
# where an F1 score reaches its best value at 1 and worst score at 0. The relative contribution of precision 
# and recall to the F1 score are equal. The formula for the F1 score is:
# F1 = 2 * (precision * recall) / (precision + recall)
from sklearn.metrics import f1_score
y_pred = pipe_logistic.predict(X_test)
f1_score(y_test, y_pred)

In [ ]:
# we can classify new messages!
msg = pd.DataFrame(columns = ["text"],
                   data    = ["The bikes are heavy and unwieldy. The collection and return of the bicycle is super-comfortable because the bikes are heavy"])

pipe_logistic.predict(msg)

In [ ]:
# we can classify new messages!
#msg = pd.DataFrame(columns = ["text"],
                   #data    = ["REMINDER FROM O2: To get 2.50 pounds free call credit and details of great offers pls reply 2 this text with your valid name, house no and postcode"])

msg = pd.DataFrame(columns = ["text"],
                   data    = ["Satisfied"])


pipe_logistic.predict(msg)

## Using bi-grams

In [ ]:
# extract features
pipe_extract = FeatureUnion([("terms", Pipeline([('clean', CleanText()), 
                                                 ('tfidf', TfidfVectorizer())])),
                             ("custom", CustomFeatures())])

# select and scale features
pipe_select_scale = Pipeline([("select", SelectKBest(score_func = chi2)),
                              ("scale", StandardScaler(with_mean = False))]) 

In [ ]:
# extract features
# you can also use bi-grams:
X_extract = pipe_extract.set_params(terms__tfidf__ngram_range = (1,2)).fit_transform(X_train, y_train)

In [ ]:
print(X_extract)

In [ ]:
# extract all features
X_select_scale = pipe_select_scale.set_params(select__k = 500).fit_transform(X_extract, y_train)
print(X_select_scale)

### Using cross-validation with parameters (grid)

In [ ]:
# Select best hyperparameters by cross validation
from sklearn.model_selection import GridSearchCV

# Model
logistic = LogisticRegression(max_iter=10000, tol=0.1, solver='lbfgs')

# Parameters: (np.logspace returns numbers spaced evenly on a log scale.)
param_logistic = {
    'C': np.logspace(-4, 4, 4)
}

# For an explanation of the 'C' parameter in scikit-learn logistic regression see:
# https://stackoverflow.com/questions/22851316/what-is-the-inverse-of-regularization-strength-in-logistic-regression-how-shoul
# C= 1/\lambda where \lambda can be assimilated to the regulatization parameter
# you probably have seen in the lasso regression
# Grid Search
cv_logistic = GridSearchCV(logistic, param_logistic, cv=10, scoring='f1')
cv_logistic.fit(X_select_scale, y_train)

In [ ]:
# See https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
print(cv_logistic.best_estimator_)

In [ ]:
print(cv_logistic.best_score_)

### Similar with pipeline

In [ ]:
# Pipe Logistic
pipe_logistic = Pipeline([('select_scale', pipe_select_scale),
                          ('classify', LogisticRegression(max_iter=10000, tol=0.1, solver='lbfgs'))])

# Parameters of pipelines can be set using ‘__’ separated parameter names:
param_logistic = {
    'classify__C': np.logspace(-4, 4, 3),
    'select_scale__select__k': [600, 1000, 5000]
}

cv_logistic = GridSearchCV(pipe_logistic, param_logistic, cv=10, scoring='f1')
cv_logistic.fit(X_extract, y_train)

In [ ]:
print(cv_logistic.best_estimator_)

In [ ]:
print(cv_logistic.best_score_)

## Other Models

### Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

# Pipe NB
pipe_nb = Pipeline([('select_scale', pipe_select_scale),
                    ('classify', MultinomialNB())])

# Parameters of pipelines can be set using ‘__’ separated parameter names:
param_nb = {
    'classify__alpha': [0.5, 1, 10],
    'select_scale__select__k': [600, 1000, 5000]
}

cv_nb = GridSearchCV(pipe_nb, param_nb, cv=10, scoring='f1')
cv_nb.fit(X_extract, y_train)
print(cv_nb.best_score_)



In [ ]:
# full pipeline
model = Pipeline([("extract", FeatureUnion([("terms", Pipeline([('clean', CleanText()), 
                                                                ('tfidf', TfidfVectorizer(ngram_range = (1,2)))])),
                                           ("custom", CustomFeatures())])),
                 ("select", SelectKBest(score_func = chi2, k = 1000)),
                 ("scale", StandardScaler(with_mean = False)),
                 ("classify", MultinomialNB())]) 

# fitting
model.fit(X_train, y_train)

# final evaluation
y_pred = model.predict(X_test)
f1_score(y_test, y_pred)

In [ ]:
# we are now able to classify new messages!
#msg = pd.DataFrame(columns = ["text"],
                   #data    = ["REMINDER FROM O2: To get 2.50 pounds free call credit and details of great offers pls reply 2 this text with your valid name, house no and postcode"])

msg = pd.DataFrame(columns = ["text"],
                   data    = ["The bikes are heavy and unwieldy. The collection and return of the bicycle is super-comfortable because the bikes are heavy"])

model.predict(msg)

In [ ]:
# we are now able to classify new messages!
#msg = pd.DataFrame(columns = ["text"],
                   #data    = ["REMINDER FROM O2: To get 2.50 pounds free call credit and details of great offers pls reply 2 this text with your valid name, house no and postcode"])

msg = pd.DataFrame(columns = ["text"],
                   data    = ["Satisfied"])


model.predict(msg)

### Support Vector Machine

In [ ]:
from sklearn.svm import LinearSVC

# Pipe SVC
pipe_svc = Pipeline([('select_scale', pipe_select_scale),
                     ('classify', LinearSVC(max_iter=10000, tol=0.1))])

# Parameters of pipelines can be set using ‘__’ separated parameter names:
param_svc = {
    'classify__C': [0.01, 0.1, 1],
    'select_scale__select__k': [600, 1000, 5000]
}

cv_svc = GridSearchCV(pipe_svc, param_svc, cv=10, scoring='f1')
cv_svc.fit(X_extract, y_train)
print(cv_svc.best_score_)

In [ ]:
# full pipeline
model = Pipeline([("extract", FeatureUnion([("terms", Pipeline([('clean', CleanText()), 
                                                                ('tfidf', TfidfVectorizer(ngram_range = (1,2)))])),
                                           ("custom", CustomFeatures())])),
                 ("select", SelectKBest(score_func = chi2, k = 1000)),
                 ("scale", StandardScaler(with_mean = False)),
                 ("classify", LinearSVC(C = 1, max_iter=10000, tol=0.1))]) 

# fitting
model.fit(X_train, y_train)

# final evaluation
y_pred = model.predict(X_test)
f1_score(y_test, y_pred)

In [ ]:
# we are now able to classify new messages!
#msg = pd.DataFrame(columns = ["text"],
                   #data    = ["REMINDER FROM O2: To get 2.50 pounds free call credit and details of great offers pls reply 2 this text with your valid name, house no and postcode"])

msg = pd.DataFrame(columns = ["text"],
                   data    = ["The bikes are heavy and unwieldy. The collection and return of the bicycle is super-comfortable because the bikes are heavy"])

model.predict(msg)

In [ ]:
# we are now able to classify new messages!
#msg = pd.DataFrame(columns = ["text"],
                   #data    = ["REMINDER FROM O2: To get 2.50 pounds free call credit and details of great offers pls reply 2 this text with your valid name, house no and postcode"])

msg = pd.DataFrame(columns = ["text"],
                   data    = ["Satisfied"])


model.predict(msg)

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Pipe RF
pipe_rf = Pipeline([('select_scale', pipe_select_scale),
                    ('classify', RandomForestClassifier())])

# Parameters of pipelines can be set using ‘__’ separated parameter names:
param_rf = {
    'classify__n_estimators': [100, 200],
    'select_scale__select__k': [600, 1000]
}

cv_rf = GridSearchCV(pipe_rf, param_rf, cv=10, scoring='f1')
cv_rf.fit(X_extract, y_train)
print(cv_rf.best_score_)

In [ ]:
# full pipeline
model = Pipeline([("extract", FeatureUnion([("terms", Pipeline([('clean', CleanText()), 
                                                                ('tfidf', TfidfVectorizer(ngram_range = (1,2)))])),
                                           ("custom", CustomFeatures())])),
                 ("select", SelectKBest(score_func = chi2, k = 1000)),
                 ("scale", StandardScaler(with_mean = False)),
                 ("classify", RandomForestClassifier())]) 

# fitting
model.fit(X_train, y_train)

# final evaluation
y_pred = model.predict(X_test)
f1_score(y_test, y_pred)

In [ ]:
# we are now able to classify new messages!
#msg = pd.DataFrame(columns = ["text"],
                   #data    = ["REMINDER FROM O2: To get 2.50 pounds free call credit and details of great offers pls reply 2 this text with your valid name, house no and postcode"])

msg = pd.DataFrame(columns = ["text"],
                   data    = ["The bikes are heavy and unwieldy. The collection and return of the bicycle is super-comfortable because the bikes are heavy"])

model.predict(msg)

In [ ]:
# we are now able to classify new messages!
#msg = pd.DataFrame(columns = ["text"],
                   #data    = ["REMINDER FROM O2: To get 2.50 pounds free call credit and details of great offers pls reply 2 this text with your valid name, house no and postcode"])

msg = pd.DataFrame(columns = ["text"],
                   data    = ["Satisfied"])


model.predict(msg)

## Long Example 1: Text clustering

In [ ]:
import re
import string
import pandas as pd

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering

## Full text for clustering

This corpus contain some strings about Google and some strings about TF-IDF from Wikipedia. Just for example

In [ ]:
all_text = """
Google and Facebook are strangling the free press to death. Democracy is the loser
Your 60-second guide to security stuff Google touted today at Next '18
A Guide to Using Android Without Selling Your Soul to Google
Review: Lenovo’s Google Smart Display is pretty and intelligent
Google Maps user spots mysterious object submerged off the coast of Greece - and no-one knows what it is
Android is better than IOS
In information retrieval, tf–idf or TFIDF, short for term frequency–inverse document frequency
is a numerical statistic that is intended to reflect how important 
a word is to a document in a collection or corpus.
It is often used as a weighting factor in searches of information retrieval
text mining, and user modeling. The tf-idf value increases proportionally
to the number of times a word appears in the document
and is offset by the frequency of the word in the corpus
""".split("\n")[1:-1]

In [ ]:
all_text

## Preprocessing and tokenizing
Firstly, we must bring every chars to lowercase and remove all punctuation, because it's not important for our task, but is very harmful for clustering algorithm. 
After that, we'll split strings to array of words.

In [ ]:
def preprocessing(line):
    line = line.lower()
    line = re.sub(r"[{}]".format(string.punctuation), " ", line)
    return line

Now, let's calculate tf-idf for this corpus

In [ ]:
tfidf_vectorizer = TfidfVectorizer(preprocessor=preprocessing)
tfidf = tfidf_vectorizer.fit_transform(all_text)

## K-means

In [ ]:
kmeans = KMeans(n_clusters=2)

In [ ]:
list(zip(kmeans.fit_predict(tfidf), all_text))

## Agglomerative Clustering

In [ ]:
hac = AgglomerativeClustering(n_clusters=2, affinity='cosine', linkage='average')

In [ ]:
list(zip(hac.fit_predict(tfidf.toarray()), all_text))

## Example 2: Topic model (1): BikeMi survey


In [ ]:
import nltk
nltk.download('wordnet')

### Cleaning and pre-processing

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop=set(stopwords.words('english'))
exclude=set(string.punctuation)
lemma=WordNetLemmatizer()
def clean(doc):
    stop_free=" ".join([i for i in doc.lower().split() if i not in stop])
    punc_free=''.join(ch for ch in stop_free if ch not in exclude)
    normalized=" ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

In [ ]:
import pandas as pd
df = pd.read_csv('Polarity2014Reduced.csv', sep = ";", header = 0)
df.columns=['review','sentiment']
df2=df[df['sentiment']==-1]
df2.shape

In [ ]:
doc_complete=df2.iloc[0:2065,0].values.tolist()
doc_clean=[clean(doc).split() for doc in doc_complete]

# Getting the document-term matrix

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
SOME_FIXED_SEED = 42
np.random.seed(SOME_FIXED_SEED)

In [ ]:
cv=CountVectorizer(min_df=2,max_df=50,ngram_range=(1,2), token_pattern=None, tokenizer=lambda doc:doc,preprocessor=lambda doc:doc)

In [ ]:
cv_features=cv.fit_transform(doc_clean)
print(cv_features.shape)
vocabulary=np.array(cv.get_feature_names())

In [ ]:
vocabulary

In [ ]:
vocabulary

# LDA ANALYSIS

In [ ]:
# Using sklearn.decomposition LDA with 11 topics
from sklearn.decomposition import LatentDirichletAllocation
TOTAL_TOPICS=11

In [ ]:
lda_model=LatentDirichletAllocation(n_components=TOTAL_TOPICS,max_iter=500,max_doc_update_iter=50,learning_method='online',batch_size=1740,learning_offset=50.,random_state=42,n_jobs=16)

In [ ]:
# Using the transformer 'fit_transform'
document_topics=lda_model.fit_transform(cv_features)


In [ ]:
document_topics.shape

In [ ]:
# Extraqcting the most important 10 terms for each topic
topic_terms=lda_model.components_
top_terms=10 # number of 'top terms'
topic_key_terms_idxs=np.argsort(-np.absolute(topic_terms), axis=1)[:,:top_terms]
topic_keyterms=vocabulary[topic_key_terms_idxs]
topics=[', '.join(topic) for topic in topic_keyterms]
pd.set_option('display.max_colwidth',-1)
topics_df=pd.DataFrame(topics,columns=['Term per Topic'], index=['Topic'+str(t) for t in range(1,TOTAL_TOPICS+1)])
topics_df

In [ ]:
dt_df=pd.DataFrame(document_topics,columns=['T'+str(i) for i in range(1,TOTAL_TOPICS+1)])
dt_df                                                                      

In [ ]:
# Column 'Contribution%' gives the max probability among the 354 
# features (terms) for each topic
dt_df=pd.DataFrame(document_topics,columns=['T'+str(i) for i in range(1,TOTAL_TOPICS+1)])
pd.options.display.float_format='{:,.5f}'.format
pd.set_option('display.max_colwidth',200)
max_contrib_topics=dt_df.max(axis=0)
dominant_topics=max_contrib_topics.index
contrib_perc=max_contrib_topics.values
document_numbers=[dt_df[dt_df[t]==max_contrib_topics.loc[t]].index[0] for t in dominant_topics]
results_df=pd.DataFrame({'Dominant Topic':dominant_topics,'Contribution%':contrib_perc, 'Answer Num': document_numbers,'Topic':topics_df['Term per Topic']})
results_df

In [ ]:
# This gives, for each topic, the % of features having prob >0.9
numT1=np.count_nonzero(dt_df['T1']>0.9)
FrT1=numT1/2133
numT2=np.count_nonzero(dt_df['T2']>0.9)
FrT2=numT2/2133
numT3=np.count_nonzero(dt_df['T3']>0.9)
FrT3=numT3/2133
numT4=np.count_nonzero(dt_df['T4']>0.9)
FrT4=numT4/2133
numT5=np.count_nonzero(dt_df['T5']>0.9)
FrT5=numT5/2133
numT6=np.count_nonzero(dt_df['T6']>0.9)
FrT6=numT6/2133
numT7=np.count_nonzero(dt_df['T7']>0.9)
FrT7=numT7/2133
numT8=np.count_nonzero(dt_df['T8']>0.9)
FrT8=numT8/2133
numT9=np.count_nonzero(dt_df['T9']>0.9)
FrT9=numT9/2133
numT10=np.count_nonzero(dt_df['T10']>0.9)
FrT10=numT10/2133
numT11=np.count_nonzero(dt_df['T11']>0.9)
FrT11=numT11/2133
d=(FrT1,FrT2,FrT3,FrT4,FrT5,FrT6,FrT7,FrT8,FrT9,FrT10,FrT11)
df_Fr=pd.DataFrame(data=d)
results_df.insert(4,'Freq 0.9-1',df_Fr.values)
results_df